In [2]:
from qiskit import QuantumCircuit, Aer, transpile, assemble
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from numpy.random import randint
import numpy as np
print("Imports Successful")

Imports Successful


In [3]:
def encode_message(bits, bases):
    message = []
    for i in range(n):
        qc = QuantumCircuit(1,1)
        if bases[i] == 0: # Prepare qubit in Z-basis
            if bits[i] == 0:
                pass 
            else:
                qc.x(0)
        else: # Prepare qubit in X-basis
            if bits[i] == 0:
                qc.h(0)
            else:
                qc.x(0)
                qc.h(0)
        qc.barrier()
        message.append(qc)
    return message

def measure_message(message, bases):
    backend = Aer.get_backend('aer_simulator')
    measurements = []
    for q in range(n):
        if bases[q] == 0: # measuring in Z-basis
            message[q].measure(0,0)
        if bases[q] == 1: # measuring in X-basis
            message[q].h(0)
            message[q].measure(0,0)
        aer_sim = Aer.get_backend('aer_simulator')
        qobj = assemble(message[q], shots=1, memory=True)
        result = aer_sim.run(qobj).result()
        measured_bit = int(result.get_memory()[0])
        measurements.append(measured_bit)
    return measurements

def remove_garbage(a_bases, b_bases, bits):
    good_bits = []
    for q in range(n):
        if a_bases[q] == b_bases[q]:
            # If both used the same basis, add
            # this to the list of 'good' bits
            good_bits.append(bits[q])
    return good_bits

In [5]:
np.random.seed()

## Super secret message
msg = "Juan Jose y Dani" #16 characters
#print('Your super secret message: ',message)

# Initial size of key
n = len(msg)*3
print("Key size: ",n)

## Step 1
# Juan generates bits
juan_bits = randint(2, size=n)

## Step 2
# Create an array to tell us which qubits
# are encoded in which bases
juan_bases = randint(2, size=n)
message = encode_message(juan_bits, juan_bases)

## Step 3
# Decide which basis to measure in:
daniel_bases = randint(2, size=n)
daniel_results = measure_message(message, daniel_bases)

## Step 4
juan_key = remove_garbage(juan_bases, daniel_bases, juan_bits)
juan_key = "".join(str(i) for i in juan_key)
daniel_key = remove_garbage(juan_bases, daniel_bases, daniel_results)
daniel_key = "".join(str(i) for i in daniel_key)

print("Size: ", len(daniel_key), "Key: ", daniel_key)

Key size:  48
Size:  24 Key:  111010011100111001100111


In [6]:
## Super secret message
mes = "Juan Jose y Dani" #16 characters
print('Your super secret message: ',mes)

Your super secret message:  Juan Jose y Dani


In [7]:
#make key same length has message
shortened_Mkey = juan_key[:len(mes)]
encoded_m=''
#encrypt message mes using encryption key final_key
for m,k in zip(mes,shortened_Mkey):
    encoded_c = chr(ord(m) + 2*ord(k) % 256)
    encoded_m += encoded_c
print('encoded message: ',encoded_m)

encoded message:  ¬×ÃÎªÏÕÇÙ¦ÃÐÉ


In [8]:
#make key same length has message
shortened_Skey = daniel_key[:len(mes)]
#decrypt message mes using encryption key final_key
result = ''
for m,k in zip(encoded_m,shortened_Skey):
    encoded_c = chr(ord(m) - 2*ord(k) % 256)
    result += encoded_c
print('recovered message:',result)

recovered message: Juan Jose y Dani
